# Turn-Based Dialogue Evaluation
* Dataset: CPsyCounE
* Model: GPT-4 Turbo

In [ ]:
import json
import os

In [ ]:
def read_json_files(folder_path):
    """
    读取指定文件夹路径下的所有json文件。
    """
    json_files = [pos_json for pos_json in os.listdir(folder_path) if pos_json.endswith('.json')]
    dialogues = []
    for json_file in json_files:
        file_path = os.path.join(folder_path, json_file)
        with open(file_path, 'r', encoding='utf-8') as f:
            dialogue_data = json.load(f)
            dialogues.append(dialogue_data)
    return dialogues


In [ ]:

def construct_turn_based_dialogues(dialogue_data):
    """
    根据对话数据构造Turn-Based对话。
    """
    turn_based_dialogues = []
    history_dialogue = ""
    for utterance in dialogue_data:
        if "求助者：" in utterance:
            if history_dialogue:
                turn_based_dialogues.append(history_dialogue)
            history_dialogue = utterance
        else:
            history_dialogue += " " + utterance
    turn_based_dialogues.append(history_dialogue)
    return turn_based_dialogues


In [ ]:

def generate_replies(turn_based_dialogues, model_reply):
    """
    使用用户提供的模型生成每个Turn-Based对话的回复。
    """
    turn_based_replies = []
    for dialogue in turn_based_dialogues:
        reply = model_reply(dialogue)
        turn_based_replies.append(reply)
    return turn_based_replies


In [ ]:

def evaluate_replies(turn_based_dialogues, turn_based_replies, evaluate_reply):
    """
    使用GPT-4评价每个Turn-Based对话回复的得分。
    """
    scores = []
    for dialogue, reply in zip(turn_based_dialogues, turn_based_replies):
        score = evaluate_reply(dialogue, reply)
        scores.append(score)
    return scores


In [40]:

def write_evaluation_results(scores, theme_folder, cnt):
    # 定位到仓库的根目录
    base_dir = os.path.abspath(os.path.join('..'))
    # 构建结果文件夹的完整路径
    results_dir = os.path.join(base_dir, "Results_Turn_Based_Dialogue_Evaluation", theme_folder)
    # 检查结果文件夹是否存在，如果不存在，则创建它
    if not os.path.exists(results_dir):
        os.makedirs(results_dir)
    
    # 完整路径
    result_file_path = os.path.join(results_dir, f"evaluation_results_{cnt}.txt")
    
    with open(result_file_path, 'w') as f:
        # 在文件开始处写入主题名
        f.write(f"{theme_folder}\n")
        # 逐行写入每轮的评分，前面带有轮数信息
        for i, score in enumerate(scores, start=1):
            f.write(f"Round {i}, Score: {score}\n")  # BUG
        # 计算并写入平均评分
        avg_score = sum(scores) / len(scores)   # BUG
        f.write(f"Average Score: {avg_score}\n")



In [ ]:
# BUG
# 示例模型回复函数（这需要替换为实际使用的模型）
def model_reply(history):
    # 这里应该是模型生成回复的代码
    # 返回生成的回复
    return "这是一个示例回复。"

# 示例评价函数（这需要替换为实际使用GPT-4的代码）
def evaluate_reply(history, reply):
    # 这里应该是GPT-4评价回复的代码
    # 返回评价得分
    return 5  # 这个分数仅为例子


## 单一主题

In [41]:
# 主题：'Career', 'Education', 'Emotion&Stress', 'Family Relationship', 'Love&Marriage', 'Mental Disease', 'Self-growth', 'Sex', 'Social Relationship'
# 主题文件夹的路径
theme_folder = '' # 填入主题文件夹的名称    
theme_folder_path = os.path.abspath(os.path.join('..', 'CPsyCounE', theme_folder))
# 执行读取JSON文件
dialogues = read_json_files(theme_folder_path)

for i in range(len(dialogues)):
    cnt = i
    dialogue_data = dialogues[i]
    # 构造Turn-Based对话
    turn_based_dialogues = construct_turn_based_dialogues(dialogue_data)
    # 生成回复
    turn_based_replies = generate_replies(turn_based_dialogues, model_reply)
    # 评价得分
    scores = evaluate_replies(turn_based_dialogues, turn_based_replies, evaluate_reply)
    # 写入评价结果
    write_evaluation_results(scores, theme_folder, cnt)

## 全部主题

In [42]:
# 9个主题文件夹的路径
folders = ['Career', 'Education', 'Emotion&Stress', 'Family Relationship', 'Love&Marriage', 'Mental Disease', 'Self-growth', 'Sex', 'Social Relationship']
for theme_folder in folders:
    theme_folder_path = os.path.abspath(os.path.join('..', 'CPsyCounE', theme_folder))
    # 执行读取JSON文件
    dialogues = read_json_files(theme_folder_path)

    for i in range(len(dialogues)):
        cnt = i
        dialogue_data = dialogues[i]
        # 构造Turn-Based对话
        turn_based_dialogues = construct_turn_based_dialogues(dialogue_data)
        # 生成回复
        turn_based_replies = generate_replies(turn_based_dialogues, model_reply)
        # 评价得分
        scores = evaluate_replies(turn_based_dialogues, turn_based_replies, evaluate_reply)
        # 写入评价结果
        write_evaluation_results(scores, theme_folder, cnt)